In [1]:
import json
import pandas as pd

In [19]:
df = pd.read_csv('operators_data.csv')

In [20]:
df.head()

,operator_address,operator_id,percentage,quorum_id,total_batches,total_unsigned_batches,timestamp
0,0x00107cfdeaddc0a3160ed2f6fedd627f313e7b1a,0x57309111be2293fbcfd77e8479a4f2570f8956aae8cd...,0.0000,0,144,0,1717934400
1,0x006b988f89579e5842bcd029955dfbfc334b6826,0x852f5c1ecf96472bb48fc8e9373b40c74ac3e2cdf32a...,0.0000,1,144,0,1717934400
2,0x01a7c2568693d65a367fde016b48c63f6673d4dc,0x7e06688af02ac562ee27c843b7e688eb1c67b469b3e6...,0.0000,1,144,0,1717934400
3,0x033bfb405e809a303df875a6e018f1a64e5dbae9,0x2e2ee953aa3c2a499640270eaaf2ba1973265f7e0d0b...,0.0625,0,144,9,1717934400
4,0x033bfb405e809a303df875a6e018f1a64e5dbae9,0x2e2ee953aa3c2a499640270eaaf2ba1973265f7e0d0b...,0.0625,1,144,9,1717934400


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13181 entries, 0 to 13180
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   operator_address        13181 non-null  object 
 1   operator_id             13181 non-null  object 
 2   percentage              13181 non-null  float64
 3   quorum_id               13181 non-null  int64  
 4   total_batches           13181 non-null  int64  
 5   total_unsigned_batches  13181 non-null  int64  
 6   timestamp               13181 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 721.0+ KB


In [22]:
# change operator address to string
df['operator_address'] = df['operator_address'].astype(str)
df['operator_id'] = df['operator_id'].astype(str)

In [23]:
len(df), len(df['operator_address'].unique())

(13181, 263)

In [24]:
# drop rows in which operator_address and timestamp are same and quorum_id is 1. keep the one in which quoram_id is 0
df = df.drop_duplicates(subset=['operator_address', 'timestamp'], keep='first')

In [25]:
len(df), len(df['operator_address'].unique())

(9732, 263)

In [26]:
# sum of all percentage of a certain operator
df['total_percentage'] = df.groupby('operator_address')['percentage'].transform('sum')

# frequency of each operator
df['frequency'] = df.groupby('operator_address')['operator_address'].transform('count')

In [27]:
df.head()

,operator_address,operator_id,percentage,quorum_id,total_batches,total_unsigned_batches,timestamp,total_percentage,frequency
0,0x00107cfdeaddc0a3160ed2f6fedd627f313e7b1a,0x57309111be2293fbcfd77e8479a4f2570f8956aae8cd...,0.0000,0,144,0,1717934400,0.019536,52
1,0x006b988f89579e5842bcd029955dfbfc334b6826,0x852f5c1ecf96472bb48fc8e9373b40c74ac3e2cdf32a...,0.0000,1,144,0,1717934400,0.010856,17
2,0x01a7c2568693d65a367fde016b48c63f6673d4dc,0x7e06688af02ac562ee27c843b7e688eb1c67b469b3e6...,0.0000,1,144,0,1717934400,0.007636,29
3,0x033bfb405e809a303df875a6e018f1a64e5dbae9,0x2e2ee953aa3c2a499640270eaaf2ba1973265f7e0d0b...,0.0625,0,144,9,1717934400,0.267125,20
5,0x047438c5ceaa6d47e8691b51f5b34c6d41a00e3d,0xdbf03be4c90917ec42220446cee2b1226ac96fd074c5...,0.0000,1,144,0,1717934400,0.000000,29


In [28]:
# order by timestamp
df = df.sort_values(by='timestamp')

In [34]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the CSV file
# df = pd.read_csv('operator_data.csv')

# Function to create lag features
def create_lag_features(df, lag=1):
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['percentage'].shift(i)
    return df

# Create lag features for the past 5 days
df = df.groupby('operator_address').apply(create_lag_features, lag=7)
df = df.dropna().reset_index(drop=True)
# print(df.head())
 
# Define features and target
# features = [col for col in df.columns if 'lag_' in col]
# last 30 days total percentage
features = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'frequency']
target = 'percentage'

# Split the data into training and testing sets
train_df = df[df['timestamp'] < 1717588800]
test_df = df[df['timestamp'] >= 1717588800]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Initialize and train the XGBoost model
model = LinearRegression()
model.fit(X_train, y_train)

/var/folders/pj/vyqltmgj79xghgpb38_96m040000gn/T/ipykernel_24264/2415128827.py:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('operator_address').apply(create_lag_features, lag=7)


LinearRegression()

In [31]:
len(X_test), len(X_train), len(df)

(1212, 4938, 6150)

In [35]:
from sklearn.metrics import mean_squared_error as MSE
# Predict the model 
pred = model.predict(X_test)
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

RMSE :  0.060309


In [37]:
pred

array([ 0.0012406 ,  0.00631149,  0.00114354, ...,  0.02295687,
       -0.00045574,  0.0012406 ])

In [40]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Define the initial types for the ONNX model
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the scikit-learn model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("lr_copy_operator.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [41]:
!giza transpile lr_copy_operator.onnx --output-path verifiable_lr

[giza][2024-06-14 16:29:31.907] No model id provided, checking if model exists ✅
[giza][2024-06-14 16:29:31.910] Model name is: lr_copy_operator
[giza][2024-06-14 16:29:35.859] Model Created with id -> 821! ✅
[giza][2024-06-14 16:29:38.748] Version Created with id -> 1! ✅
[giza][2024-06-14 16:29:38.749] Sending model for transpilation ✅ 
[giza][2024-06-14 16:30:16.307] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠼ Transpiling Model...
[giza][2024-06-14 16:30:18.168] Downloading model ✅
[giza][2024-06-14 16:30:18.179] model saved at: verifiable_lr


In [145]:
!giza endpoints deploy --model-id 821 --version-id 1

▰▰▰▱▱▱▱ Creating endpoint!t!
[giza][2024-06-14 16:07:17.910] Endpoint is successful ✅
[giza][2024-06-14 16:07:17.913] Endpoint created with id -> 372 ✅
[giza][2024-06-14 16:07:17.914] Endpoint created with endpoint URL: https://endpoint-ashq-814-2-3e6340fe-7i3yxzspbq-ew.a.run.app 🎉


In [146]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,quorum_id,total_batches,total_unsigned_batches,frequency,total_percentage
6654,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,144,0,52,0.558237
6655,0.006944,0.0,0.0,0.0,0.069444,0.027778,0.013889,0,144,0,34,0.273075
6656,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,144,0,47,0.289335
6657,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,144,0,53,0.075797
6658,0.000000,0.0,0.0,0.0,0.006944,0.000000,0.000000,1,144,0,27,0.017800


In [ ]:
['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'quorum_id', 'total_batches', 'total_unsigned_batches', 'frequency', 'total_percentage', 'timestamp']

In [147]:
input = np.array(X_test).astype(np.float32)

In [148]:
input

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 5.20000000e+01, 5.58237314e-01],
       [6.94444450e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.40000000e+01, 2.73074806e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.70000000e+01, 2.89335161e-01],
       ...,
       [2.08333340e-02, 1.02459015e-02, 0.00000000e+00, ...,
        0.00000000e+00, 4.50000000e+01, 3.58770514e+00],
       [0.00000000e+00, 6.14754111e-03, 0.00000000e+00, ...,
        0.00000000e+00, 3.00000000e+01, 2.56230216e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 5.20000000e+01, 1.95358880e-02]], dtype=float32)

In [150]:
from giza.agents.model import GizaModel
from sklearn.metrics import mean_squared_error as MSE

MODEL_ID = 814  # Update with your model ID
VERSION_ID = 2  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True
    )

    return result, proof_id

def execution():
    # The input data type should match the model's expected input
    # input = np.array([[0.980643, 0.979649, 0.975971, 0.974366, 0.970623, 0.965317, 0.979038, 875.450182]]).astype(np.float32)

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(
        f"Predicted value for input {input.flatten()[0]} is {result[0].flatten()[0]}")
    
    rmse = np.sqrt(MSE(y_test, result))
    print("RMSE : % f" %(rmse))

    return result, proof_id


execution()

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input 0.0 is 0.0018768310546875
RMSE :  0.055565


(array([[ 0.00187683],
        [ 0.00891113],
        [ 0.00047302],
        ...,
        [ 0.02735901],
        [ 0.00798035],
        [-0.00047302]]),
 '74e3dbf7881944f383ab4f8601de28eb')

In [151]:
latest_date = df['timestamp'].max()
unique_ids = df['operator_address'].unique()
operators = df[df['timestamp']==latest_date]

In [152]:
lag_features = operators[features].values
input = lag_features.astype(np.float32)
model = GizaModel(id=814, version=2)
(result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True, job_size='L'
)
print("Result: ", result, " Proof Id: ",  proof_id)

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Result:  [[-1.83105469e-04]
 [ 8.40759277e-03]
 [-4.27246094e-04]
 [ 9.35806274e-01]
 [-1.31225586e-03]
 [ 8.66699219e-03]
 [-5.03540039e-04]
 [ 1.15966797e-03]
 [ 3.92150879e-02]
 [ 4.73022461e-04]
 [-4.11987305e-04]
 [ 1.83105469e-04]
 [-5.79833984e-04]
 [-7.21740723e-03]
 [ 1.77001953e-03]
 [ 8.39233398e-03]
 [-1.69372559e-03]
 [ 8.13293457e-03]
 [ 1.28173828e-03]
 [-1.73950195e-03]
 [-7.32421875e-04]
 [ 2.86254883e-02]
 [ 1.19323730e-02]
 [-2.28881836e-04]
 [ 9.03320312e-03]
 [ 1.00708008e-02]
 [-3.81469727e-04]
 [ 1.87683105e-03]
 [ 1.28479004e-02]
 [-1.06811523e-04]
 [ 2.10571289e-02]
 [ 1.84173584e-02]
 [ 8.63647461e-03]
 [ 5.23376465e-03]
 [ 8.71276855e-03]
 [ 8.31604004e-03]
 [ 4.25720215e-03]
 [ 8.33129883e-03]
 [ 8.71276855e-03]
 [-1.37329102e-04]
 [ 9.35806274e-01]
 [ 8.66699219e-03]
 [ 8.20922852e-03]
 [ 6.56127930e-04]
 [ 1.22680664e-02]
 [ 8.68225098e-03]
 [-4.57763672e-04]
 [ 8.36181641e-03]
 [ 8.75854

In [155]:
# find max id
operator = operators.iloc[np.argmax(result.flatten())]
print(f"Operator with the highest predicted percentage: {operator['operator_address']} with a predicted percentage of {result.flatten().min()}")

Operator with the highest predicted percentage: 0x0713cd9686cd160bbe2f4376c5adee44d0e2ed18 with a predicted percentage of -0.0072174072265625
